In [1]:
import json
with open("hotpot_dev_distractor_v1.json") as f:
    datasets = json.load(f)

In [2]:
datasets = datasets[:500]

In [3]:
from dotenv import load_dotenv
from openai import OpenAI
import os

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def llm_create(prompt):
        messages = [{"role": "user", "content": prompt}]
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
        )
        return completion.choices[0].message.content

In [8]:
find_support_fact_prompts = """You are given a question and a list of evidence document.
Your task is to find the support fact from the evidence document for the question.
Some questions require a combination of several documents to understand the cause and effect
Output doc_name and doc_id if relative to the question.

Question:{question}

Document:{docs}

Output following json format, example:
```json
{{
    "Scott Derrickson":[0, 1],
    "Ed Wood":[0]
}}
```"""

answer_prompt = """Base on the following document and answer the question.
Only short answer is needed, no explanation required.

Question:{question}

Document:{docs}
"""

In [7]:
from long_memory.hotpot_component import HotPotWeaviateLongMemory

long_mem = HotPotWeaviateLongMemory()

Detect existed deafult user group memory space, loading...
Detect existed deafult user child memory space, loading...


In [6]:
import pandas as pd

df = pd.DataFrame()
df['question_id'] = None
df['answer'] = None
df['org_res_from_long_mem'] = None

In [9]:
import pandas as pd
df = pd.read_json("hotpot_df_v1.json", lines=True)

In [22]:
import re

error_list = []
for i in range(400, 500):
    try:
        print(f"---{i}---")
        sp_fact = []
        long_mem.del_memory()
        doc_list = datasets[i]["context"]
        question_id = datasets[i]["_id"]
        df.loc[i, 'question_id'] = question_id 
        question = datasets[i]["question"]
        for doc in doc_list:
            long_mem.add_hotpot_doc(doc)
        res = long_mem.get_memory(question, recall=True)
        df.loc[i, 'org_res_from_long_mem'] = str(res)
        answer_res = llm_create(answer_prompt.format(question=question, docs=res))
        df.loc[i, 'answer'] = answer_res
        
        """response = llm_create(find_support_fact_prompts.format(question=question, docs=res))
        json_res = json.loads(re.search(r"```json(.*?)```", response, re.DOTALL).group(1).strip())
        for doc_name in json_res:
            for doc_id in json_res[doc_name]:
                sp_fact.append([doc_name, int(doc_id)])
        df.loc[i, 'support_fact'] = str(sp_fact)"""
    except:
        print(f"-----error-----")
        error_list.append(i)
        
    if (i+1) % 10 == 0:
        df.to_json("hotpot_df.json", orient="records", lines=True)
        

---400---
Detect empty group memory, create memory space...
Detect empty child memory, create memory space...
---401---
Detect empty group memory, create memory space...
Detect empty child memory, create memory space...
---402---
Detect empty group memory, create memory space...
Detect empty child memory, create memory space...
---403---
Detect empty group memory, create memory space...
Detect empty child memory, create memory space...
---404---
Detect empty group memory, create memory space...
Detect empty child memory, create memory space...
---405---
Detect empty group memory, create memory space...
Detect empty child memory, create memory space...
---406---
Detect empty group memory, create memory space...
Detect empty child memory, create memory space...
---407---
Detect empty group memory, create memory space...
Detect empty child memory, create memory space...
---408---
Detect empty group memory, create memory space...
Detect empty child memory, create memory space...
---409---


In [27]:
error_list

[496]

處理這些 error list 跑 hotpot 評分

In [31]:
import re

fix_list = [496]
error_list = []
for i in fix_list:
    try:
        print(f"---{i}---")
        sp_fact = []
        long_mem.del_memory()
        doc_list = datasets[i]["context"]
        question_id = datasets[i]["_id"]
        df.loc[i, 'question_id'] = question_id 
        question = datasets[i]["question"]
        for doc in doc_list:
            long_mem.add_hotpot_doc(doc)
        res = long_mem.get_memory(question, recall=True)
        df.loc[i, 'org_res_from_long_mem'] = str(res)
        answer_res = llm_create(answer_prompt.format(question=question, docs=res))
        df.loc[i, 'answer'] = answer_res
        
        """response = llm_create(find_support_fact_prompts.format(question=question, docs=res))
        json_res = json.loads(re.search(r"```json(.*?)```", response, re.DOTALL).group(1).strip())
        for doc_name in json_res:
            for doc_id in json_res[doc_name]:
                sp_fact.append([doc_name, int(doc_id)])
        df.loc[i, 'support_fact'] = str(sp_fact)"""
    except Exception as e:
        print(f"-----error:{e}-----")
        error_list.append(i)
df.to_json("hotpot_df_v1.json", orient="records", lines=True)

---496---
Detect empty group memory, create memory space...
Detect empty child memory, create memory space...


In [32]:
df.to_json("hotpot_df_v1.json", orient="records", lines=True)

In [33]:
df

,question_id,answer,org_res_from_long_mem
0,5a8b57f25542995d1e6f1371,Yes.,"{'search times': 1, 'used queries': ['Were Sco..."
1,5a8c7595554299585d9e36b6,U.S. ambassador to Ghana and Czechoslovakia.,"{'search times': 2, 'used queries': ['What gov..."
2,5a85ea095542994775f606a8,Animorphs,"{'search times': 1, 'used queries': ['What sci..."
3,5adbf0a255429947ff17385a,No.,"{'search times': 1, 'used queries': ['Are the ..."
4,5a8e3ea95542995a26add48d,Greenwich Village.,"{'search times': 1, 'used queries': ['The dire..."
...,...,...,...
495,5a79b7f6554299029c4b5f6f,"4,613","{'search times': 1, 'used queries': ['How many..."
496,5ab626d555429953192ad279,"Newark, New Jersey.","{'search times': 4, 'used queries': ['Anthony ..."
497,5a84873e5542997175ce1eec,137th edition,"{'search times': 1, 'used queries': [""What edi..."
498,5ac537975542996feb3fea3c,Ronald Reagan,"{'search times': 1, 'used queries': ['Walt Zeb..."
